# Scrape the LangChain documentation into a ChromaDB Vector Database and use it for a GPT-4 chatbot  to talk with it!

In this notebook, I will introduce you to vector databases. I will:
- Web scrape the LangChain documentation
- Store the LangChain documentation in a Chroma DB vector database
- Create a retriever to retrieve the desired information
- Create a Q&A chatbot with GPT-4
- Show how you can delete and reopen a vector database locally to save space
Visualise your vector database (very cool, read till the end!)

This notebook is connected to a medium article: [Medium articles](https://medium.com/@rubentak)

ref: https://medium.com/@rubentak/unleashing-the-power-of-intelligent-chatbots-with-gpt-4-and-vector-databases-a-step-by-step-8027e2ce9e78

In [27]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_urls_from_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        urls = set()

        for link in soup.find_all('a', href=True):
            absolute_url = urljoin(url, link['href'])
            urls.add(absolute_url)

        return urls
    else:
        print(f"Failed to fetch the page: {response.status_code}")
        return []

In [31]:
target_url = "https://docs.oracle.com/en/cloud/get-started/index.html"  # Replace with the URL you want to scrape
found_urls = extract_urls_from_page(target_url)

print("Found URLs:")
for url in found_urls:
    print(url)

Found URLs:
https://docs.oracle.com/en/cloud/get-started/subscriptions-cloud/get-trial-or-subscription.html
https://docs.cloud.oracle.com/iaas/Content/home.htm
https://docs.oracle.com/index.html
https://docs.oracle.com/en/cloud/marketplace/index.html
https://docs.oracle.com/pls/topic/lookup?ctx=en/legal&id=cpyr
https://www.oracle.com/cloud/free/
https://docs.oracle.com/pls/topic/lookup?ctx=en/legal&id=about
https://docs.oracle.com/en/cloud/cloud-at-customer/index.html
https://docs.oracle.com/en/cloud/get-started/index.html
https://docs.oracle.com/en/cloud/paas/index.html
https://docs.oracle.com/pls/topic/lookup?ctx=en/legal&id=privacy
https://docs.oracle.com/en/cloud/saas/index.html
https://docs.oracle.com/
https://docs.oracle.com/en/cloud/index.html
https://docs.oracle.com/en/cloud/get-started/subscriptions-cloud/index.html
https://docs.oracle.com/pls/topic/lookup?ctx=en/legal&id=contact


In [34]:
import os
def save_content(link_list):
    for i, link in enumerate(link_list):
        html_data = get_data(link)
        soup = BeautifulSoup(html_data, "html.parser")
        text = soup.get_text()

        # Remove the first 835 lines
        lines = text.splitlines()
        cleaned_text = "\n".join(lines)

        # Get the first 3 words in the cleaned text
        words = cleaned_text.split()[:3]
        file_name_prefix = "_".join(words)

        # Replace special characters and spaces with an underscore
        file_name_prefix = re.sub(r"[^a-zA-Z0-9]+", "_", file_name_prefix)

        # Get the current working directory
        current_dir = os.getcwd()

        # Move up one level to the parent directory
        parent_dir = os.path.dirname(current_dir)

        # Set the path to the data folder
        data_folder = os.path.join(parent_dir, "data/langchain_doc")

        # Create the data folder if it doesn't exist
        if not os.path.exists(data_folder):
            os.makedirs(data_folder)

        # Set the path to the output file
        output_file = os.path.join(data_folder, f"{i}_{file_name_prefix}.txt")

        # Save the cleaned content to the output file
        with open(output_file, "w") as f:
            f.write(cleaned_text)

In [35]:
# save the content of the links into txt files
save_content(found_urls)

# Q&A bot with langchain over a directory

In [8]:
# Import libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

In [10]:
# Create a new openai api key
os.environ["OPENAI_API_KEY"] = "sk-..."
# set up openai api key
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [11]:
# Print number of txt files in directory
loader = DirectoryLoader('/Users/erictak/PycharmProjects/langchain/data/langchain_doc', glob="./*.txt")
doc = loader.load ( )
len(doc)

679

In [12]:
# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(doc)

In [13]:
# Count the number of chunks
len(texts)

5576

In [14]:
# Print the first chunk
texts[0]

Document(page_content='Twitter\n\nContents\n\nInstallation and Setup\n\nDocument Loader\n\nTwitter#\n\nTwitter is an online social media and social networking service.\n\nInstallation and Setup#\n\npip install tweepy\n\nWe must initialize the loader with the Twitter API token, and we need to set up the Twitter username.\n\nDocument Loader#\n\nSee a usage example.\n\nfrom langchain.document_loaders import TwitterTweetLoader\n\nprevious\n\nTrello\n\nnext\n\nUnstructured\n\nContents\n\nInstallation and Setup\n\nDocument Loader\n\nBy Harrison Chase\n\n© Copyright 2023, Harrison Chase.\n\nLast updated on Jun 13, 2023.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/langchain_doc/592_Twitter_Contents_Installation.txt'})

# Data base creation with ChromaDB

https://www.youtube.com/watch?v=3yPBVii7Ct0

In [15]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

# OpenAI embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [16]:
# Persist the db to disk
vectordb.persist()
vectordb = None

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [17]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


# Create retriever

In [41]:
retriever = vectordb.as_retriever()

In [42]:
docs = retriever.get_relevant_documents("What to do when getting started?")

In [43]:
docs

[Document(page_content='Step 1: Create Tools# Agents are largely defined by the tools they can use. If you have a specific task you want the agent to accomplish, you have to give it access to the right tools. We have many tools natively in LangChain, so you should first look to see if any of them meet your needs. But we also make it easy to define a custom tool, so if you need custom tools you should absolutely do that.\n\n(Optional) Step 2: Modify Agent# The built-in LangChain agent types are designed to work well in generic situations, but you may be able to improve performance by modifying the agent implementation. There are several ways you could do this:\n\nModify the base prompt. This can be used to give the agent more context on how it should behave, etc. Modify the output parser. This is necessary if the agent is having trouble parsing the language model output.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/langchain_doc/644_Agents_Contents_Create.txt'}),

In [44]:
len(docs)

4

In [45]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [46]:
retriever.search_type

'similarity'

In [47]:
retriever.search_kwargs

{'k': 2}

# Create a question answering chain

In [49]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [50]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [51]:
# Question
query = "What are the steps of the Quickstart Guide?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 Step 1: Create Tools (Optional), Step 2: Modify Agent (Optional), Step 3: Modify Agent Executor.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/426_Agents_Contents_Create.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/644_Agents_Contents_Create.txt


In [53]:
# Break it down
query = "What are all agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
#llm_response



> Entering new RetrievalQA chain...

> Finished chain.
 Custom Agent, Custom LLM Agent, Custom LLM Agent (with a ChatModel), Custom MRKL Agent, Custom MultiAction Agent, Custom Agent with Tool Retrieval, Conversation Agent (for Chat Models), Conversation Agent MRKL, MRKL Chat, ReAct, Self Ask With Search, Structured Tool Chat Agent.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/382_Agents_Agents_Note.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/382_Agents_Agents_Note.txt


In [54]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f8dba0c1ff0>)

In [55]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Deleteing the DB

In [250]:
!zip -r db.zip ./db

updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/index/ (stored 0%)
updating: db/index/index_metadata_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 5%)
updating: db/index/id_to_uuid_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 37%)
updating: db/index/uuid_to_id_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 39%)
updating: db/index/index_b9a5e02f-ebd0-4b13-8858-b30b211c4546.bin (deflated 17%)
updating: db/index/index_d80886e4-65e1-4231-8c73-99ff58d68061.bin (deflated 17%)
updating: db/index/uuid_to_id_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 41%)
updating: db/index/id_to_uuid_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 32%)
updating: db/index/index_metadata_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 5%)
updating: db/chroma-collections.parquet (deflated 50%)
updating: db/.DS_Store (deflated 96%)


In [251]:
# To clean up, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# Delete the directory
!rm -rf db/

# Starting again loading the db

In [57]:
!unzip db.zip

Archive:  db.zip
replace db/chroma-embeddings.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

In [59]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

Using embedded DuckDB with persistence: data will be stored in: db


#### Usung turbo GPT API

In [60]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [61]:
# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [62]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [63]:
# Question
query = "What are the agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
There are two main types of agents mentioned in the context: Action Agents and Plan-and-Execute Agents. Action Agents decide the actions to take and execute those actions one at a time, while Plan-and-Execute Agents first decide a plan of actions to take, and then execute those actions one at a time.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/639_Agents_Contents_Action.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/344_Agents_Contents_Action.txt


In [64]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [65]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


# Visualizing the Vector db
https://github.com/mtybadger/chromaviz?ref=reactjsexample.com

https://github.com/avantrio/chroma-viewer


In [ ]:
from chromaviz import visualize_collection
visualize_collection(vectordb._collection)